## 234 - Orderings Points to Properly Plot Polygons

[Youtube](https://www.youtube.com/watch?v=uPHPVj8pTc4)

In [ ]:
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

In [ ]:
df = pd.read_csv('airsig.txt', skiprows = 5)
df

In [ ]:
df.iloc[3]['lon:lat points']

In [ ]:
def make_lat_lon_from_points_text(text):
	""" Takes text string from AWC and returns lat lon lists. """
	
	points = text.split(';')
	lons, lats = [], []
	for pair in points:
		lon, lat = pair.split(':')
		lons.append(float(lon))
		lats.append(float(lat))
		
	return lats, lons

In [ ]:
def make_polygons_for_airmet_sigmet(fname):
	""" Take AWC AIRMET/SIGMET text and create polygons. """
	df = pd.read_csv(fname, skiprows = 5)
	df = df[df['hazard'] == 'IFR']
	polygons = []
	for row in df.itertools():
		row = row[1]
		lats, lons = make_lat_lon_from_points_text(row['lon:lat points'])
		p = list(zip(lons, lats))
		p = order_points(p)
		p = Polygon(p)
		polygons.append(p)
	return polygons

In [ ]:
def order_points(points):
	""" Order points in a counterclockwise way for polygons. """
	
	points = np.array(points)
	
	# Compute Center
	center_x, center_y = np.mean(points[:, 0]), np.mean(points[:, 1])
	
	# Sort by angle
	polar_angles = np.arctan2((points[:, 1] - center_y), (points[:, 0] - center_x))
	sort_idx = np.argsort(polar_angles)
	return points[sort_idx]	

In [ ]:
polygons = make_polygon_for_airmet_sigmet('airsig.txt')

In [ ]:
map_crs = ccrs.LambertConformal(central_longitude = -100, central_latitude = 45)
bounds = [(-122, -75, 25, 50)]

In [ ]:
fig = plt.figure(figsize = (17, 12))
ax = fig.add_subplot(1, 1, 1, projection = map_crs)
ax.set_extent(*bounds, crs = ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m))
ax.add_feature(cfeature.STATES, linewidth = 0.5)
for p in polygons:
	ax.add_geometries([p], crs = ccrs.PlateCarree(), facecolor = 'None', edgecolor = 'tab:brown', linewidth = 3)	